# Rick_TSP

Working example of a genetic Travelling Salesman Problem.

Rick Comments:
- I feel as though I have reached a point of deminishing returns with this project. Although there is still SO much room for improvement, I think that my time would be better spent applying my learning to a new project (I have another optimization problem in mind. It is a replenishment order optimizer for one of my clients, I have already built the tool with Solver in Excel, and I think it is very well suited to Python.)

In [ ]:
import matplotlib.pyplot as plt
from rick_tsp import RickTSP

In [ ]:
rick = RickTSP()
rick.create_towns(random=1, size=10)
rick.generate_population(size=6)
rick.generate_TFM()
population_fitness = rick.breed(num_gen=500)

In [ ]:
print('Your resulting population is:', rick.population_size)   
print('The route lengths associated with your population are:\n', population_fitness)
print('\nThis plot shows you the distance of the best known route after a given number of itterations')
plt.plot(fittest_known)
plt.title('Distance of best candidate')
plt.ylabel('Distance of tour')
plt.xlabel('Number of itterations completed')
ax = plt.show